In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import unicodedata

In [2]:
url = 'https://www.cdc.gov/coronavirus/2019-ncov/cases-updates/cases-in-us.html'

### Extract date

In [3]:
content = requests.get(url).content
soup =BeautifulSoup(content, 'lxml')

In [4]:
def _removeNonAscii(s): return "".join(i for i in s if ord(i)<128)

data = soup.findAll(text=re.compile(".*pdated .*202"))
data


['Last updated on May 6, 2020']

In [5]:
if len(data)>0:
    m = re.search(".*pdated .* (\w+ \d+, 202\d)", _removeNonAscii(data[0]))
    data_date_str = m.group(1)
    data_date = datetime.strptime(data_date_str, '%B %d, %Y').strftime('%Y-%m-%d')
else:
    data_date = datetime.utcnow().strftime('%Y-%m-%d')
data_date

'2020-05-06'

### Extract data and add into Dataframe

In [6]:
dfs = pd.read_html(url)

In [7]:
if len(dfs) == 0 :
    raise Exception('There is no dataframes detected')

In [8]:
dfs

[                           Unnamed: 0_level_0  \
                             Age group (years)   
 0                                      Totals   
 1                    Race missing/unspecified   
 2                              Race specified   
 3             Among those with race specified   
 4            American Indian or Alaska Native   
 5                                       Asian   
 6                   Black or African American   
 7   Native Hawaiian or other Pacific Islander   
 8                                       White   
 9                              Multiple/other   
 10                                        NaN   
 11              Ethnicity missing/unspecified   
 12                        Ethnicity specified   
 13       Among those with ethnicity specified   
 14                            Hispanic/Latino   
 15                        Non-Hispanic/Latino   
 
                No. of cases (% of total)  \
                                     < 18   
 0      

In [9]:
df = dfs[0]
df

Unnamed: 0_level_0  \
                            Age group (years)   
0                                      Totals   
1                    Race missing/unspecified   
2                              Race specified   
3             Among those with race specified   
4            American Indian or Alaska Native   
5                                       Asian   
6                   Black or African American   
7   Native Hawaiian or other Pacific Islander   
8                                       White   
9                              Multiple/other   
10                                        NaN   
11              Ethnicity missing/unspecified   
12                        Ethnicity specified   
13       Among those with ethnicity specified   
14                            Hispanic/Latino   
15                        Non-Hispanic/Latino   

               No. of cases (% of total)  \
                                    < 18   
0                                  21195   
1                          13,857(65.4%)   
2                           7,338(34.6%)   
3        Among those with race specified   
4                              256(3.5%)   
5                              342(4.7%)   
6                           1,674(22.8%)   
7                               51(0.7%)   
8                           3,610(49.2%)   
9                           1,405(19.1%)   
10                                   NaN   
11                         13,414(63.3%)   
12                          7,781(36.7%)   
13  Among those with ethnicity specified   
14                          3,121(40.1%)   
15                          4,660(59.9%)   

                                          \
                                   18-44   
0                                 322703   
1                         197,011(61.1%)   
2                         125,692(38.9%)   
3        Among those with race specified   
4                            1,683(1.3%)   
5                            6,673(5.3%)   
6                          34,349(27.3%)   
7                              524(0.4%)   
8                          62,387(49.6%)   
9                          20,076(16.0%)   
10                                   NaN   
11                        203,300(63.0%)   
12                        119,403(37.0%)   
13  Among those with ethnicity specified   
14                         39,065(32.7%)   
15                         80,338(67.3%)   

                                          \
                                   45-64   
0                                 310779   
1                         177,403(57.1%)   
2                         133,376(42.9%)   
3        Among those with race specified   
4                            1,394(1.0%)   
5                            7,110(5.3%)   
6                          40,889(30.7%)   
7                              407(0.3%)   
8                          65,908(49.4%)   
9                          17,668(13.2%)   
10                                   NaN   
11                        188,263(60.6%)   
12                        122,516(39.4%)   
13  Among those with ethnicity specified   
14                         31,983(26.1%)   
15                         90,533(73.9%)   

                                          \
                                   65-74   
0                                  96149   
1                          45,658(47.5%)   
2                          50,491(52.5%)   
3        Among those with race specified   
4                              361(0.7%)   
5                            2,331(4.6%)   
6                          15,806(31.3%)   
7                              112(0.2%)   
8                          26,983(53.4%)   
9                            4,898(9.7%)   
10                                   NaN   
11                         52,358(54.5%)   
12                         43,791(45.5%)   
13  Among those with ethnicity specified   
14                          7,978(18.2%)   
15               

In [10]:
csv_file='../data/cdc-us-cases-by-races-and-age-group.csv'
df.to_csv(csv_file, index=False)

In [11]:
# col0 = df.columns[0]
# df = df.set_index(col0).T
# df



In [12]:
# # remove non alphanumeric characters from the columns
# # also lowercase them
# df.columns = df.columns.map(lambda x: re.sub('\W+', '', str.lower(x)))

# # add column date to the front
# df.insert(0, 'date', data_date)


### Save to file (old data)


In [13]:
# from pathlib import Path
# csv_file = '../data/cdc-us-cases.csv'

# if Path(csv_file).exists():
#     # read out the old dat
#     old_df = pd.read_csv(csv_file)
#     df = df.append(old_df)
# df.drop_duplicates(subset=['date'], inplace=True)

In [14]:
# df.sort_values(by=['date'], inplace=True)
# df
# df.to_csv(csv_file, index=False)